In [2]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
print("Load a ply point cloud, print it, and render it")
# 加载数据（按H查看命令）
pcd = o3d.io.read_point_cloud("fragment.ply")
print(pcd)
print(np.asarray(pcd.points))
# 可视化
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

Load a ply point cloud, print it, and render it
PointCloud with 196133 points.
[[0.65234375 0.84686458 2.37890625]
 [0.65234375 0.83984375 2.38430572]
 [0.66737998 0.83984375 2.37890625]
 ...
 [2.00839925 2.39453125 1.88671875]
 [2.00390625 2.39488506 1.88671875]
 [2.00390625 2.39453125 1.88793314]]


In [4]:
# 体素降采样
print("Downsample the point cloud with a voxel of 0.05.")
downpcd = pcd.voxel_down_sample(voxel_size=0.05)
o3d.visualization.draw_geometries([downpcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

Downsample the point cloud with a voxel of 0.05.


In [5]:
# 顶点法向量估计
print("Recompute the normal of the downsampled point cloud")
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([downpcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024],
                                  point_show_normal=True)

Recompute the normal of the downsampled point cloud


In [6]:
# 裁剪点云
print("Load a polygon volume and use it to crop the original point cloud")
pcd = o3d.io.read_point_cloud("fragment.ply")
# 读取一个指定多边形选择区域的json文件
vol = o3d.visualization.read_selection_polygon_volume("cropped.json")
# 过滤掉点。只留下椅子。
chair = vol.crop_point_cloud(pcd)
o3d.visualization.draw_geometries([chair],
                                  zoom=0.7,
                                  front=[0.5439, -0.2333, -0.8060],
                                  lookat=[2.4615, 2.1331, 1.338],
                                  up=[-0.1781, -0.9708, 0.1608])

Load a polygon volume and use it to crop the original point cloud


In [7]:
# 上色
print("Paint chair")
# 将所有的点涂成一个统一的颜色。颜色是在RGB空间，[0, 1]范围内
chair.paint_uniform_color([1, 0.9, 0])
o3d.visualization.draw_geometries([chair],
                                  zoom=0.7,
                                  front=[0.5439, -0.2333, -0.8060],
                                  lookat=[2.4615, 2.1331, 1.338],
                                  up=[-0.1781, -0.9708, 0.1608])

Paint chair


In [8]:
# 点云距离
# 计算从源点云到目标点云的距离。也就是说，它为源点云中的每个点计算与目标点云中最近的点的距离。
# Load data
pcd = o3d.io.read_point_cloud("fragment.ply")
vol = o3d.visualization.read_selection_polygon_volume("cropped.json")
chair = vol.crop_point_cloud(pcd)

dists = pcd.compute_point_cloud_distance(chair)
dists = np.asarray(dists)
ind = np.where(dists > 0.01)[0]
pcd_without_chair = pcd.select_by_index(ind)
o3d.visualization.draw_geometries([pcd_without_chair],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

In [9]:
# 包围体积
# 无方向
aabb = chair.get_axis_aligned_bounding_box()
aabb.color = (1, 0, 0)
# 有方向
obb = chair.get_oriented_bounding_box()
obb.color = (0, 1, 0)
o3d.visualization.draw_geometries([chair, aabb, obb],
                                  zoom=0.7,
                                  front=[0.5439, -0.2333, -0.8060],
                                  lookat=[2.4615, 2.1331, 1.338],
                                  up=[-0.1781, -0.9708, 0.1608])

In [10]:
# 凸面体 凸面体是包含所有点的最小的凸面集
mesh = o3d.io.read_triangle_mesh("BunnyMesh.ply")
mesh.compute_vertex_normals()

pcl = mesh.sample_points_poisson_disk(number_of_points=2000)
hull, _ = pcl.compute_convex_hull()
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([pcl, hull_ls])

In [12]:
# DBSCAN聚类
import matplotlib.pyplot as plt

pcd = o3d.io.read_point_cloud("fragment.ply")
# 设置 Open3D 的日志输出级
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    # eps定义了集群中与邻居的距离，min_points定义了形成一个集群所需的最小点数。该函数返回标签，其中标签-1表示噪声。
    # 返回每个点的簇标签
    labels = np.array(pcd.cluster_dbscan(eps=0.02, min_points=10, print_progress=True))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.455,
                                  front=[-0.4999, -0.1659, -0.8499],
                                  lookat=[2.1813, 2.0619, 2.0999],
                                  up=[0.1204, -0.9852, 0.1215])

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 10
point cloud has 10 clusters


In [14]:
pdc=o3d.io.read_triangle_mesh("fragment.ply")
# distance_threshold定义了一个点与一个估计的平面的最大距离，以便被认为是一个离群点；
# ransac_n定义了为估计一个平面而随机采样的点的数量；
# num_iterations定义了一个随机平面被采样和验证的频率。
plane_model,inliers=pcd.segment_plane(distance_threshold=0.01,ransac_n=3,num_iterations=1000)
[a,b,c,d]=plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
inlier_cloud=pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0,0,0])
outlier_cloud=pcd.select_by_index(inliers,invert=True)
o3d.visualization.draw_geometries([inlier_cloud,outlier_cloud],
                                  zoom=0.8,
                                  front=[-0.4999, -0.1659, -0.8499],
                                  lookat=[2.1813, 2.0619, 2.0999],
                                  up=[0.1204, -0.9852, 0.1215])

[Open3D WARNING] geometry::TriangleMesh appears to be a geometry::PointCloud (only contains vertices, but no triangles).
Plane equation: -0.00x + 1.00y + 0.02z + -2.43 = 0


In [16]:
# 隐藏点去除算法
mesh=o3d.io.read_triangle_mesh("ArmadilloMesh.ply")
mesh.compute_vertex_normals()

pcd=mesh.sample_points_poisson_disk(number_of_points=5000)
diameter=np.linalg.norm(np.asarray(pcd.get_max_bound())-np.asarray(pcd.get_min_bound()))
o3d.visualization.draw_geometries([pcd])

In [17]:
print("Define parameters used for hidden_point_removal")
camera = [0, 0, diameter]
radius = diameter * 100

print("Get all points that are visible from given view point")
_, pt_map = pcd.hidden_point_removal(camera, radius)

print("Visualize result")
pcd = pcd.select_by_index(pt_map)
o3d.visualization.draw_geometries([pcd])

Define parameters used for hidden_point_removal
Get all points that are visible from given view point
Visualize result


In [20]:
# 从点云构建KDTree
pcd=o3d.io.read_point_cloud("fragment.ply")
pcd.paint_uniform_color([0.5,0.5,0.5])
pcd_tree=o3d.geometry.KDTreeFlann(pcd)

# 寻找相邻的点
print("Paint the 1501st point red.")
pcd.colors[1500] = [1, 0, 0]

print("Find its 200 nearest neighbors, and paint them blue.")
[k, idx, _] = pcd_tree.search_knn_vector_3d(pcd.points[1500], 200)
np.asarray(pcd.colors)[idx[1:], :] = [0, 0, 1]

print("Visualize the point cloud.")
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.5599,
                                  front=[-0.4958, 0.8229, 0.2773],
                                  lookat=[2.1126, 1.0163, -1.8543],
                                  up=[0.1007, -0.2626, 0.9596])

Paint the 1501st point red.
Find its 200 nearest neighbors, and paint them blue.
Visualize the point cloud.
